## HQC

In [1]:
%run ProductCode.ipynb
%run ConcatenatedCode.ipynb

In [2]:
class HQC:
    
    def __init__(self, w, w_e, w_r, C):
        
        self.w = w
        self.w_e = w_e
        self.w_r = w_r
        self.C = C
        
        self.n = self.C.n
        self.k = self.C.k
        self.delta = self.C.delta
        
        self.F = GF(2)
        self.R = PolynomialRing(self.F, 'x'); x = self.R.gen()
        self.S = QuotientRing(self.R, x**self.n - 1, 'a'); a = self.S.gen()
        
        self.pk, self.sk = self.KeyGen()
        
        
    def KeyGen(self):
        
        h = self.R.random_element(degree=(0,self.n))
        h = self.S(h)
        
        x = [0] * self.n
        x = self.RandomGenWithWeight(x, self.w)
        x = self.S(x)
        
        y = [0] * self.n
        y = self.RandomGenWithWeight(y, self.w)
        y = self.S(y)
        
        s = x + h * y
        
        pk = (h, s)
        sk = (x, y)
        
        return pk, sk
    
    def Encrypt(self, m):
        
        u = []
        v = []
        
        for i in range(0, len(m), self.k):
            u_current, v_current = self.EncryptBlock(m[i:i+self.k])
            u.extend(u_current)
            v.extend(v_current)
            
        return (self.S(u), self.S(v))
            
    
    def EncryptBlock(self, m):
        
        if len(m) != self.k:
            raise ValueError('wrong size of message')
        
        e = [0] * self.n
        e = self.RandomGenWithWeight(e, self.w_e)
        e = self.S(e)
        
        r1 = [0] * self.n
        r1 = self.RandomGenWithWeight(r1, self.w_r)
        r1 = self.S(r1)
        
        r2 = [0] * self.n
        r2 = self.RandomGenWithWeight(r2, self.w_r)
        r2 = self.S(r2)
        
        #errors = (self.sk[0] * r2 - r1 * self.sk[1] + e).list()
        #errors_HW  = errors.count(1)
        #print('HW(x * r2 - r1 * y + e) = ', errors_HW)
        #print('delta = ', self.delta)
        
        u = r1 + self.pk[0] * r2
        
        v = self.C.Encoding(m, out = 'int')
        v = self.S(v)
        #error_weight = (self.pk[1] * r2 + e).list()
        #error_HW  = error_weight.count(1)
        #print('HW(s*r2 + e) = ', error_HW)
        v = v + self.pk[1] * r2 + e
        
        return (u,v)
    
    def Decrypt(self, sk, c):
        
        d = c[1] - c[0] * sk[1]
        d = d.list()
        
        if len(d) != self.C.code_n:
            d = d[:-(self.n - self.C.code_n)]
        
        return self.C.Decoding(d)
        
        
    def RandomGenWithWeight(self, array, weight):

        for i in range(weight):
            cur = ZZ.random_element(0,len(array))
            while array[cur] == 1:
                cur = ZZ.random_element(0,len(array))
            
            array[cur] = 1
        
        return array

In [3]:
C = HQC(w = 5, w_e = 5, w_r = 5, C = ProductCode(BCHCode(n = 15, b = 1, D = 7), RepetitionCode(n = 5)))

In [4]:
print(C.pk)

(a^59 + a^58 + a^56 + a^55 + a^53 + a^52 + a^50 + a^48 + a^46 + a^45 + a^44 + a^43 + a^42 + a^41 + a^39 + a^38 + a^37 + a^30 + a^29 + a^28 + a^27 + a^26 + a^25 + a^24 + a^23 + a^21 + a^18 + a^17 + a^16 + a^14 + a^10 + a^8 + a^7 + a^5 + a^3, a^74 + a^73 + a^71 + a^70 + a^64 + a^63 + a^62 + a^61 + a^60 + a^59 + a^56 + a^55 + a^54 + a^53 + a^52 + a^51 + a^50 + a^49 + a^47 + a^46 + a^42 + a^40 + a^37 + a^35 + a^30 + a^29 + a^27 + a^24 + a^23 + a^22 + a^21 + a^19 + a^18 + a^16 + a^12 + a^11 + a^10 + a^6 + a^4 + a^3 + a^2 + 1)


In [5]:
print(C.sk)

(a^70 + a^57 + a^44 + a^31 + a^30, a^60 + a^57 + a^48 + a^27 + a^3)


In [6]:
m = []
for i in range(C.k):
    elem = ZZ.random_element(0,2)
    m.append(elem)
    
m = vector(C.F, m)
print(m)

(0, 0, 0, 1, 0)


In [7]:
m = '10101'

In [8]:
c = C.Encrypt(m)
print(c)

(a^74 + a^73 + a^72 + a^71 + a^70 + a^68 + a^67 + a^63 + a^62 + a^60 + a^59 + a^54 + a^53 + a^50 + a^49 + a^48 + a^46 + a^41 + a^40 + a^38 + a^37 + a^32 + a^30 + a^28 + a^27 + a^26 + a^24 + a^23 + a^21 + a^18 + a^17 + a^12 + a^10 + a^9 + a^7 + a^6 + a^2 + a, a^73 + a^70 + a^69 + a^68 + a^66 + a^62 + a^61 + a^60 + a^59 + a^58 + a^56 + a^55 + a^52 + a^51 + a^50 + a^49 + a^48 + a^47 + a^44 + a^42 + a^39 + a^35 + a^34 + a^32 + a^31 + a^28 + a^26 + a^24 + a^23 + a^22 + a^21 + a^20 + a^18 + a^17 + a^16 + a^15 + a^14 + a^13 + a^12 + a^11 + a^10 + a^9 + a^7 + a^5 + a^3 + a)


In [9]:
d = code.Decrypt(code.sk, c)
d == m

NameError: name 'code' is not defined